<a href="https://colab.research.google.com/github/ALLEN0607/lumaa_recommender/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn

print("pandas", pd.__version__)
print("numpy", np.__version__)
print("sklearn", sklearn.__version__)

pandas 2.2.2
numpy 1.26.4
sklearn 1.6.1


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Full Dataset
df_movies = pd.read_csv("data/tmdb_5000_movies.csv")
# Small Dataset
#df_movies = df_movies.sample(n=500, random_state=42).reset_index(drop=True)

df_movies.info()

print(df_movies.head(n=10))
print(df_movies.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
import ast

# 解析并拼接 genres, keywords
def parse_json_column(json_str):
  try:
    data_list = ast.literal_eval(json_str)   # string -> python object(list or dict)
    names = [d['name'] for d in data_list]
    return " ".join(names)
  except:
    return ""

# 给 df_movies 增加解析列
df_movies["parsed_genres"] = df_movies["genres"].apply(parse_json_column)
df_movies["parsed_keywords"] = df_movies["keywords"].apply(parse_json_column)

# combine overview, genres, keywords
df_movies["combined"] = (
    df_movies["overview"].fillna("") + " " +
    df_movies["parsed_genres"].fillna("") + " " +
    df_movies["parsed_keywords"].fillna("")
)

print(df_movies["combined"])

0      When the Switchblade, the most sophisticated p...
1      In a futuristic London, the rising sea levels ...
2      On a beautiful college campus, something ugly ...
3      Two girlfriends on a summer holiday in Spain b...
4      As Harry begins his sixth year at Hogwarts, he...
                             ...                        
495    The Big Apple's in big trouble, as indestructi...
496    Dr. John Dolittle the beloved doctor is back, ...
497    Orson Welles, as judge Rauch, holds a lengthy ...
498    Everyone deserves a chance to follow their dre...
499    An amphibious shark-like monster terrorizes an...
Name: combined, Length: 500, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# stop_words(the, is, and, ...)
vectorizer = TfidfVectorizer(stop_words='english')
# 对 combined 列做向量化
tfidf_matrix = vectorizer.fit_transform(df_movies["combined"])

# Print tfidf_martix shape (num_movies, num_features)
print("TF-IDF matrix shape: ", tfidf_matrix.shape)


TF-IDF matrix shape:  (500, 7014)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 推荐电影
def recommend_movies(user_input, df, vectorizer, tfidf_matrix, top_n=5):
  # 将user_input转成TF-IDE向量
  user_vec = vectorizer.transform([user_input])

  similarity_scores = cosine_similarity(user_vec, tfidf_matrix)

  scores = similarity_scores.flatten()

  top_indices = scores.argsort()[::-1][:top_n]

  res = []
  for idx in top_indices:
    movie_title = df.iloc[idx]["title"]
    score = scores[idx]
    res.append((movie_title, score))

  return res

In [ ]:
# test case
user_query = "I love thrilling action movies set in space, with a comedic twist."

recommendations = recommend_movies(
    user_query,
    df_movies,
    vectorizer,
    tfidf_matrix,
    top_n=5
)

for title, score in recommendations:
    print(f"{title}: {score:.4f}")


Big Hero 6: 0.1442
Babe: Pig in the City: 0.1307
Red Planet: 0.1207
Son of the Mask: 0.1171
Cinderella Man: 0.1160
